<a href="https://colab.research.google.com/github/RyanStoutGent/MSMP-B/blob/main/CSass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data and packages

In [234]:
!pip install textdistance
!pip install strsimpy
import textdistance
from strsimpy.qgram import QGram
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

In [235]:
import json
import pandas as pd
import numpy as np
import re
import sympy
import random
import math
import copy
import nltk
import sys
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from sklearn.cluster import AgglomerativeClustering
from itertools import combinations
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Cleaning

In [236]:
def remove_spaces_tokens(text):
    return re.sub(r'^[\s\W]+', '', text.group(0))

In [237]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'\((.*?)\)', r'\1', text)
  placeholder = "UNIQUE_PLACEHOLDER"

  to_inch = [r'\bInch\b', r'inches', r'["”]', r'-inch', r' inch', r'inch']
  for variant in to_inch:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'inch')

  to_hz = [r'Hertz', r'hertz', r'Hz', r'HZ', r' hz', r'-hz', r'hz']
  for variant in to_hz:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'hz')

  to_lbs = [r'lbs\.',r'lb\.', r'lbs', r'lb', r'pounds', r'pound']
  for variant in to_lbs:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'lbs')

  to_nit = [r'nit', r' cd/mÂ²', r'cd/mÂ²', r' cd/m²', r'cd/m²', r' cd/m2', r'cd/m2', r'Nit' r'cd/mÂ²', r'cd/mâ²']
  for variant in to_nit:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'nit')

  to_hrs = [r'hours', r'hour', r'hrs', r'Hrs (typical)', r'hrs (typical)']
  for variant in to_hrs:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'hrs')

  to_w = [r'watts', r'watt', r'w']
  for variant in to_w:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'w')

  to_mw = [r'mw', r' mw', r'Mw', r'MW', r' Mw']
  for variant in to_mw:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'mw')

  text = re.sub(r'[\s\W]*(inch|hz|wifi|mw|lbs|hrs|w|nit)\b', remove_spaces_tokens, text)

  return text

In [238]:
def clean_features_title(input_file,output_file):
  with open(input_file, 'r') as file:
    tv_data = copy.deepcopy(json.load(file))

  for model_id, tvs in tv_data.items():
    for tv in tvs:
      if 'title' in tv:
        tv['title'] = clean_text(tv['title'])
      if 'featuresMap' in tv:
        for feature, value in tv['featuresMap'].items():
          cleaned_feature = clean_text(value)
          tv['featuresMap'][feature] = cleaned_feature

  with open(output_file, 'w') as file:
      json.dump(tv_data, file, indent=4)

  return output_file

## Binary Matrix

In [239]:
def get_brand_dictionary(tvs):

  all_brand_names = set()
  for tv in tvs:
    features_map = tv.get('featuresMap', {})
    brand = features_map.get('Brand') or features_map.get('Brand Name') or features_map.get('Brand Name:')
    if brand:
      all_brand_names.add(brand)

  return list(all_brand_names)

In [240]:
def get_mw_title(title):

  mw_pattern = r'([a-zA-Z0-9]*(([0-9]+[^0-9, ]+)|([^0-9, ]+[0-9]+))[a-zA-Z0-9]*)'
  regex = re.compile(mw_pattern)
  model_words = [match[0] for match in regex.findall(title)]

  return model_words

In [241]:
def get_title_dictionary(titles_list):
  titles_dictionary = set()

  for title in titles_list:
    mw_title = get_mw_title(title)
    titles_dictionary.update(mw_title)

  return list(titles_dictionary)

In [242]:
def get_brand_tv(tv, brand_names):

  title  = tv['title']
  features_map = tv.get('featuresMap', {})
  brand = features_map.get('Brand') or features_map.get('Brand Name') or features_map.get('Brand Name:')

  if brand:
    brand_name = brand
  else:
    for name in brand_names:
      if name in title.lower():
        brand_name = name
        break
      else:
        brand_name = 'unknown'

  return brand_name

In [243]:
def get_mw_feature(feature):
  #unit_list = ['inch','hz','wifi','mw','lbs','nit','w','hrs']
  unit_list = []
  feature_pattern = r'(^\d+(\.\d+)?[a-zA-Z]*$|^\d+(\.\d+)?$)'
  regex = re.compile(feature_pattern)
  matches = regex.findall(feature)

  model_words = []
  tokens = feature.split()
  for token in tokens:
    matches = regex.findall(token)
    for match in matches:
      full_match = match[0]


      if any(full_match.endswith(unit) for unit in unit_list):
        model_words.append(full_match)

      elif re.match(r'^\d+(\.\d+)?', full_match):
        numeric_part = re.match(r'^\d+(\.\d+)?', full_match).group()
        model_words.append(numeric_part)

  return list(model_words)

In [244]:
def get_feature_dictionary(features_maps):
  features_dictionary = set()

  for feature_map in features_maps:
    for feature, value in feature_map.items():
      mw_feature = get_mw_feature(value)
      features_dictionary.update(mw_feature)

  return list(features_dictionary)

In [245]:
def get_tv_dictionary(tv,brand_dictionary):

  title = tv['title']
  feature_map = tv.get('featuresMap', {})
  brand = set(get_brand_tv(tv, brand_dictionary))

  tv_dictionary_title = set(get_mw_title(title))

  tv_dictionary_feature = set()
  for feature, value in feature_map.items():
    mw_feature = get_mw_feature(value)
    tv_dictionary_feature.update(mw_feature)

  tv_dictionary = set(tv_dictionary_title.union(tv_dictionary_feature).union(brand))

  return list(tv_dictionary)

In [246]:
def get_binary_matrix(all_tvs):

  tv_titles = [tv['title'] for tv in all_tvs]
  brand_dictionary = set(get_brand_dictionary(all_tvs))
  title_dictionary = set(get_title_dictionary(tv_titles))
  feature_maps = [tv.get('featuresMap', {}) for tv in all_tvs]
  feature_dictionary = set(get_feature_dictionary(feature_maps))
  dictionary = set(brand_dictionary.union(title_dictionary).union(feature_dictionary))

  binary_matrix = np.zeros((len(dictionary), len(all_tvs)), dtype=int)

  for tv_index, tv in enumerate(all_tvs):
    feature_map = tv.get('featuresMap', {})
    title = tv['title']
    brand = get_brand_tv(tv, brand_dictionary)
    tv_dictionary = set(get_tv_dictionary(tv, brand_dictionary))
    for word_index, word in enumerate(dictionary):
      if word in tv_dictionary:
        binary_matrix[word_index, tv_index] = 1

  return binary_matrix

## Minhashing

In [247]:
def hash_function(a, b, p, x):
  return (a * x + b) % p

In [248]:
def get_hashes(n_rows, n_hashes):
  n_hashes = n_hashes
  p = sympy.nextprime(n_rows)

  hash_functions = []
  for i in range(n_hashes):
    a = random.randint(1, p - 1)
    b = random.randint(0, p - 1)
    hash_functions.append((a, b, p))

  return hash_functions

In [249]:
def minhashing(binary_matrix, n_hashes):
  n_rows, n_cols = binary_matrix.shape
  hash_functions = get_hashes(n_rows, n_hashes)
  n_hashes = len(hash_functions)

  signature_matrix = np.full((n_hashes, n_cols), float('inf'))

  for j in range(n_cols):
    for i in range(n_rows):
      if binary_matrix[i,j] == 1:
        for k in range(n_hashes):
          a, b, p = hash_functions[k]
          hash_value = hash_function(a, b, p, i)
          signature_matrix[k, j] = min(signature_matrix[k, j], hash_value)

  return signature_matrix

## LSH

In [250]:
def lsh(signature_matrix, b,r):
  n_hashes, n_cols = signature_matrix.shape

  if int(n_hashes) != b * r:
    raise ValueError("The number of rows in signature matrix must be equal to b * r.")

  buckets = {}
  candidate_pairs = set()

  for i in range(b):
    for j in range(n_cols):
      start_band = i * r
      end_band = (i + 1) * r
      signature_band = tuple(signature_matrix[start_band:end_band, j])
      buckets.setdefault(signature_band, []).append(j)

  for bucket in buckets.values():
    bucket = list(set(bucket))
    if len(bucket) > 1:
      for i in range(len(bucket)):
        for j in range(i + 1, len(bucket)):
          pair = tuple(sorted((bucket[i], bucket[j])))
          candidate_pairs.add(pair)

  return list(candidate_pairs)

## Baseline code

In [251]:
def clean_text_old(text):
  text = text.lower()
  text = re.sub(r'\((.*?)\)', r'\1', text)
  placeholder = "UNIQUE_PLACEHOLDER"

  to_inch = [r'Inch', r'inches', r'["”]', r'-inch', r' inch', r'inch']
  for variant in to_inch:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'inch')

  to_hz = [r'Hertz', r'hertz', r'Hz', r'HZ', r' hz', r'-hz', r'hz']
  for variant in to_hz:
    text = re.sub(variant, placeholder, text)
  text = text.replace(placeholder, 'hz')

  text = re.sub(r'[\s\W]*(inch|hz)\b', remove_spaces_tokens, text)

  return text

In [252]:
def clean_features_title_old(input_file,output_file):
  with open(input_file, 'r') as file:
    tv_data = copy.deepcopy(json.load(file))

  for model_id, tvs in tv_data.items():
    for tv in tvs:
      if 'title' in tv:
        tv['title'] = clean_text_old(tv['title'])
      if 'featuresMap' in tv:
        for feature, value in tv['featuresMap'].items():
          cleaned_feature = clean_text_old(value)
          tv['featuresMap'][feature] = cleaned_feature

  with open(output_file, 'w') as file:
      json.dump(tv_data, file, indent=4)

  return output_file

In [253]:
def get_tv_dictionary_old(tv):

  title = tv['title']
  feature_map = tv.get('featuresMap', {})
  tv_dictionary_title = set(get_mw_title(title))

  tv_dictionary_feature = set()
  for feature, value in feature_map.items():
    mw_feature = get_mw_feature(value)
    tv_dictionary_feature.update(mw_feature)

  tv_dictionary = set(tv_dictionary_title.union(tv_dictionary_feature))

  return list(tv_dictionary)

In [254]:
def get_binary_matrix_old(all_tvs):

  tv_titles = [tv['title'] for tv in all_tvs]
  title_dictionary = set(get_title_dictionary(tv_titles))
  feature_maps = [tv.get('featuresMap', {}) for tv in all_tvs]
  feature_dictionary = set(get_feature_dictionary(feature_maps))
  dictionary = set(title_dictionary.union(feature_dictionary))

  binary_matrix = np.zeros((len(dictionary), len(all_tvs)), dtype=int)

  for tv_index, tv in enumerate(all_tvs):
    feature_map = tv.get('featuresMap', {})
    title = tv['title']
    tv_dictionary = set(get_tv_dictionary_old(tv))
    for word_index, word in enumerate(dictionary):
      if word in tv_dictionary:
        binary_matrix[word_index, tv_index] = 1

  return binary_matrix

## MSM

In [255]:
def get_qGramSimilarity(string1, string2, q):

  qgram = QGram(q)
  distance = (len(string1) + len(string2) - qgram.distance(string1, string2)) / (len(string1) + len(string2))

  return(distance)

In [256]:
def get_feature_similarity(tv1, tv2, q, gamma):

  feature_map1 = tv1.get('featuresMap', {})
  keys1 = set(feature_map1.keys())
  feature_map2 = tv2.get('featuresMap', {})
  keys2 = set(feature_map2.keys())

  total_similarity = 0
  weighted_key_similarity = 0
  num_matches = 0

  unmatched_keys_1 = set(keys1)
  unmatched_keys_2 = set(keys2)

  for key_1, value_1 in feature_map1.items():
    for key_2, value_2 in feature_map2.items():
      key_similarity = get_qGramSimilarity(key_1, key_2, q)

      if key_similarity >= gamma:
        value_similarity = get_qGramSimilarity(value_1, value_2, q)
        total_similarity += key_similarity * value_similarity
        weighted_key_similarity += key_similarity
        num_matches += 1

        if key_1 in unmatched_keys_1:
          unmatched_keys_1.remove(key_1)
        if key_2 in unmatched_keys_2:
          unmatched_keys_2.remove(key_2)

  matching_similarity = 0
  if weighted_key_similarity > 0:
    matching_similarity = total_similarity / weighted_key_similarity

  unmactched_values1 = [feature_map1[key] for key in unmatched_keys_1]
  unmactched_values2 = [feature_map2[key] for key in unmatched_keys_2]

  model_words1 = set()
  for value in unmactched_values1:
    model_words1.update(get_mw_feature(value))
  model_words2 = set()
  for value in unmactched_values2:
    model_words2.update(get_mw_feature(value))

  non_matching_similarity = 0
  if model_words1 or model_words2:
    non_matching_similarity = len(model_words1.intersection(model_words2)) / len(model_words1.union(model_words2))

  return matching_similarity, non_matching_similarity, num_matches

In [257]:
def get_average_LS_similarity(X,Y):

  numerator = 0
  denominator = 0

  for x in X:
    for y in Y:
      numerator += (1 - textdistance.levenshtein.normalized_distance(x, y)) * (len(x) + len(y))
      denominator += len(x) + len(y)

  return(numerator / denominator)

In [258]:
def split_mw(model_word):

    non_numeric = re.sub(r'[0-9]', '', model_word)
    numeric = re.sub(r'[^0-9]', '', model_word)

    return non_numeric, numeric

In [259]:
def get_title_similarity(title1, title2, brand_dictionary, alpha, beta, delta, epsilon):

  noisy_words = set(['best', 'buy', 'newegg.com', 'thenerdsnet', 'refurbished', 'open', 'box'])
  words_to_remove = set(stopwords.words('english')).union(noisy_words)

  clean_title1 = re.sub(r'[^a-zA-Z0-9. ]', ' ', title1)
  clean_title2 = re.sub(r'[^a-zA-Z0-9. ]', ' ', title2)
  clean_title1 = set([word for word in clean_title1.split(' ') if word not in words_to_remove])
  clean_title2 = set([word for word in clean_title2.split(' ') if word not in words_to_remove])

  titleCosine = len(clean_title1.intersection(clean_title2)) / (math.sqrt(len(clean_title1)) * math.sqrt(len(clean_title2)))

  if titleCosine >= alpha:
    return 1

  #Compute weighted sim
  mw_title1 = get_mw_title(title1)
  mw_title2 = get_mw_title(title2)

  for mw_i in mw_title1:
    for mw_j in mw_title2:
      non_numeric_i, numeric_i = split_mw(mw_i)
      non_numeric_j, numeric_j = split_mw(mw_j)
      if textdistance.levenshtein.normalized_distance(non_numeric_i, non_numeric_j) < epsilon and numeric_i != numeric_j:
        return -1

  similarity_title = beta * titleCosine + (1 - beta) * get_average_LS_similarity(clean_title1, clean_title2)

  similar = False
  num = 0
  denom = 0
  for mw_i in mw_title1:
    for mw_j in mw_title2:
      non_numeric_i, numeric_i = split_mw(mw_i)
      non_numeric_j, numeric_j = split_mw(mw_j)

      if numeric_i == numeric_j and textdistance.levenshtein.normalized_distance(non_numeric_i, non_numeric_j) < epsilon:
        similar = True
        num += (1 - textdistance.levenshtein.normalized_distance(mw_i, mw_j)) * len(mw_i) * len(mw_j)
        denom += len(mw_i) + len(mw_j)

  if similar:
    ls_similarity_titles = num/denom
    similarity_title = delta * ls_similarity_titles + (1 - delta) * similarity_title

  return similarity_title

In [260]:
def get_dissimilarity_matrix(tvs, candidate_pairs, q, alpha, beta, gamma, delta, mu, epsilon):

  matrix = np.ones((len(tvs), len(tvs))) * 10000000000
  brand_dictionary = get_brand_dictionary(tvs)

  for pair in candidate_pairs:
    tv_i = tvs[pair[0]]
    tv_j = tvs[pair[1]]
    brand_i = get_brand_tv(tv_i, brand_dictionary)
    brand_j = get_brand_tv(tv_j, brand_dictionary)

    if tv_i['shop'] != tv_j['shop'] and brand_i == brand_j:
      matchingkey_similarity, non_matchingkey_similarity, num_matches = get_feature_similarity(tv_i, tv_j, q, gamma)
      title_similarity = get_title_similarity(tv_i['title'], tv_j['title'], brand_dictionary, alpha, beta, delta, epsilon)

      min_num_features = min(len(tv_i.get('featuresMap', {})), len(tv_j.get('featuresMap', {})))
      if title_similarity == -1:
        theta1 = num_matches / min_num_features if min_num_features > 0 else 0
        theta2 = 1 - theta1
        total_similarity = theta1 * matchingkey_similarity + theta2 * non_matchingkey_similarity
      else:
        theta1 = (1-mu) * (num_matches) / min_num_features if min_num_features > 0 else 0
        theta2 = 1 - theta1 - mu
        total_similarity = theta1 * matchingkey_similarity + theta2 * non_matchingkey_similarity + mu * title_similarity

      total_dissimilarity = 1 - total_similarity
      matrix[pair[0], pair[1]] = total_dissimilarity
      matrix[pair[1], pair[0]] = total_dissimilarity

  return matrix

In [261]:
def hClustering(matrix, epsilon):

  cluster = AgglomerativeClustering(
      n_clusters=None,
      linkage='single',
      distance_threshold=epsilon,
      metric='precomputed'
  )
  cluster_labels = cluster.fit_predict(matrix)

  clusters = {}
  for tv, cluster in enumerate(cluster_labels):
    clusters.setdefault(cluster, []).append(tv)

  duplicate_pairs = set()
  for tvs in clusters.values():
    if len(tvs) > 1:
      for pair in combinations(tvs, 2):
        duplicate_pairs.add(tuple(sorted(pair)))

  return list(duplicate_pairs)

##Metrics

In [262]:
def get_true_duplicates(all_tvs):

  true_duplicates = set()

  for i in range(len(all_tvs)):
    for j in range(i + 1, len(all_tvs)):
      if all_tvs[i]['modelID'] == all_tvs[j]['modelID']:
        true_duplicates.add(tuple(sorted((i, j))))

  return true_duplicates

In [263]:
def get_pq_pc_f1star(true_pairs, candidate_pairs):

  df = len(set(candidate_pairs).intersection(set(true_pairs)))
  nc = len(set(candidate_pairs))
  dn = len(set(true_pairs))

  pq = df / nc if nc > 0 else 0
  pc = df / dn if dn > 0 else 0

  f1star = 0
  if pc + pq > 0:
    f1star = 2 * pc * pq / (pc + pq)

  return pq, pc, f1star

In [264]:
def get_pr_re_F1(true_pairs, predicted_pairs,candidate_pairs):
  possible_trues = set(true_pairs).intersection(set(candidate_pairs))

  tp = len(set(possible_trues).intersection(set(predicted_pairs)))
  fn = len(set(possible_trues)) - tp
  fp = len(set(predicted_pairs)) - tp

  precision = tp / (tp + fp) if (tp + fp) > 0 else 0
  recall = tp / (tp + fn) if (tp + fn) > 0 else 0

  f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

  return precision, recall, f1

In [265]:
def get_pr_re_F1_total(true_pairs, predicted_pairs,candidate_pairs):

  tp = len(set(true_pairs).intersection(set(predicted_pairs)))
  fn = len(set(true_pairs)) - tp
  fp = len(set(predicted_pairs)) - tp

  precision = tp / (tp + fp) if (tp + fp) > 0 else 0
  recall = tp / (tp + fn) if (tp + fn) > 0 else 0

  f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

  return precision, recall, f1

##Bootstrapping

In [273]:
def bootstrapLSH(all_tvs, n_hashes, iterations, old):
  random.seed(1)

  n_train = 1024
  n_test = 600
  n_products = len(all_tvs)

  def getIterationResults(tvs,sigmatrix,n_hashes):
    results_iter = []
    for b in range(5, int(n_hashes//2 + 1)):
      if n_hashes % b != 0:
        continue
      r = n_hashes // b
      candidate_pairs = lsh(sigmatrix, b, r)
      true_pairs = get_true_duplicates(tvs)
      comparison_fraction = len(candidate_pairs) / total_comparisons
      pq, pc, f1star = get_pq_pc_f1star(true_pairs, candidate_pairs)
      results_iter.append([b, r, comparison_fraction, pq, pc, f1star])

    return results_iter

  results_total = []
  for i in range(iterations):
    print('iteration: ', i+1)
    train_indices = sorted(np.random.choice(n_products, size=n_train, replace=True))
    train_tvs = [all_tvs[i] for i in train_indices]
    n_tvs = len(train_tvs)
    total_comparisons = n_tvs * (n_tvs - 1) // 2

    if old:
      binary_matrix_train = get_binary_matrix_old(train_tvs)
    else:
      binary_matrix_train = get_binary_matrix(train_tvs)

    sigmatrix_train = minhashing(binary_matrix_train, n_hashes)
    iteration_results = getIterationResults(train_tvs,sigmatrix_train, n_hashes)
    results_total.extend(iteration_results)

  results_total = np.array(results_total)

  results_dict = defaultdict(list)
  for row in results_total:
    b, r, comparison_fraction, pq, pc, f1star = row
    results_dict[(int(b), int(r))].append([comparison_fraction, pq, pc, f1star])

  averaged_results = []
  for (b, r), metrics in results_dict.items():
    metrics_array = np.array(metrics)
    avg_metrics = metrics_array.mean(axis=0)
    averaged_results.append([b, r] + avg_metrics.tolist())

  averaged_results = np.array(averaged_results)

  return averaged_results

In [274]:
def bootstrapMSM(all_tvs, n_hashes, n_bootsraps, old):
  random.seed(1)

  alpha = 0.602
  beta = 0.00
  gamma = 0.756
  delta = 0.7
  mu = 0.65
  epsilon = 0.522
  distance_tresholds = [i/20 for i in range(1,20)]
  q = 3

  n_train = 1024
  n_test = 600
  n_products = len(all_tvs)

  results_total = []
  for i in range(n_bootsraps):
    print('iteration: ', i+1)
    train_indices = sorted(np.random.choice(n_products, size=n_train, replace=True))
    all_indices = set(range(n_products))
    test_indices = sorted(list(all_indices - set(train_indices)))
    train_tvs = [all_tvs[i] for i in train_indices]
    test_tvs = [all_tvs[i] for i in test_indices]

    if old:
      train_binary = get_binary_matrix_old(train_tvs)
      test_binary = get_binary_matrix_old(test_tvs)
    else:
      train_binary = get_binary_matrix(train_tvs)
      test_binary = get_binary_matrix(test_tvs)
    train_signature = minhashing(train_binary, n_hashes)

    n_test_tvs = len(test_tvs)
    total_comparisons_test = n_test_tvs * (n_test_tvs - 1) // 2
    test_signature = minhashing(test_binary,n_hashes)

    results_iter = []
    for b in range(5, int(n_hashes//3 + 1)):
      if n_hashes % b != 0:
        continue
      r = n_hashes // b
      train_candidate_pairs = lsh(train_signature, int(b), int(r))
      train_true_pairs = get_true_duplicates(train_tvs)
      train_similarity_matrix = get_dissimilarity_matrix(train_tvs, train_candidate_pairs, q, alpha, beta, gamma, delta, mu, epsilon)

      train_best_f1 = -np.inf
      for treshold in distance_tresholds:
        train_predicted_pairs = hClustering(train_similarity_matrix, treshold)
        train_precision, train_recall, train_f1 = get_pr_re_F1(train_true_pairs, train_predicted_pairs, train_candidate_pairs)
        a,c, train_f1_total = get_pr_re_F1_total(train_true_pairs, train_predicted_pairs, train_candidate_pairs)
        if train_f1 > train_best_f1:
          train_best_f1 = train_f1
          best_treshold = treshold

      test_candidate_pairs = lsh(test_signature, int(b), int(r))
      test_similarity_matrix = get_dissimilarity_matrix(test_tvs, test_candidate_pairs, q, alpha, beta, gamma, delta, mu, epsilon)
      test_predicted_pairs = hClustering(test_similarity_matrix, best_treshold)
      test_true_pairs = get_true_duplicates(test_tvs)
      test_precision, test_recall, test_f1 = get_pr_re_F1(test_true_pairs, test_predicted_pairs, test_candidate_pairs)
      a,c, test_f1_total = get_pr_re_F1_total(test_true_pairs, test_predicted_pairs, test_candidate_pairs)

      test_comparison_fraction = len(test_candidate_pairs) / total_comparisons_test
      results_iter.append([b, r, test_comparison_fraction, test_precision, test_recall, test_f1,test_f1_total])

    results_total.extend(results_iter)

  results_total = np.array(results_total)
  results_dict = defaultdict(list)

  for row in results_total:
    b, r, test_comparison_fraction, test_precision, test_recall,test_f1, test_f1_total = row
    results_dict[(int(b), int(r))].append([test_comparison_fraction, test_precision, test_recall, test_f1,test_f1_total])

  averaged_results = []
  for (b, r), metrics in results_dict.items():
    metrics_array = np.array(metrics)
    avg_metrics = metrics_array.mean(axis=0)
    averaged_results.append([b, r] + avg_metrics.tolist())

  averaged_results = np.array(averaged_results)

  return averaged_results

In [275]:
def bootstrapMSMtotal(all_tvs, n_hashes, n_bootsraps, old):
  random.seed(1)

  alpha = 0.602
  beta = 0.00
  gamma = 0.756
  delta = 0.7
  mu = 0.65
  epsilon = 0.522
  distance_tresholds = [i/20 for i in range(1,20)]
  q = 3

  n_train = 1024
  n_test = 600
  n_products = len(all_tvs)

  results_total = []
  for i in range(n_bootsraps):
    print('iteration: ', i+1)
    train_indices = sorted(np.random.choice(n_products, size=n_train, replace=True))
    all_indices = set(range(n_products))
    test_indices = sorted(list(all_indices - set(train_indices)))
    train_tvs = [all_tvs[i] for i in train_indices]
    test_tvs = [all_tvs[i] for i in test_indices]

    if old:
      train_binary = get_binary_matrix_old(train_tvs)
      test_binary = get_binary_matrix_old(test_tvs)
    else:
      train_binary = get_binary_matrix(train_tvs)
      test_binary = get_binary_matrix(test_tvs)
    train_signature = minhashing(train_binary, n_hashes)

    n_test_tvs = len(test_tvs)
    total_comparisons_test = n_test_tvs * (n_test_tvs - 1) // 2
    test_signature = minhashing(test_binary,n_hashes)

    results_iter = []
    for b in range(5, int(n_hashes//4 + 1)):
      if n_hashes % b != 0:
        continue
      r = n_hashes // b
      train_candidate_pairs = lsh(train_signature, int(b), int(r))
      train_true_pairs = get_true_duplicates(train_tvs)
      train_similarity_matrix = get_dissimilarity_matrix(train_tvs, train_candidate_pairs, q, alpha, beta, gamma, delta, mu, epsilon)

      train_best_f1 = -np.inf
      for treshold in distance_tresholds:
        train_predicted_pairs = hClustering(train_similarity_matrix, treshold)
        train_precision, train_recall, train_f1 = get_pr_re_F1(train_true_pairs, train_predicted_pairs, train_candidate_pairs)
        a,c, train_f1_total = get_pr_re_F1_total(train_true_pairs, train_predicted_pairs, train_candidate_pairs)
        if train_f1_total > train_best_f1:
          train_best_f1 = train_f1_total
          best_treshold = treshold

      test_candidate_pairs = lsh(test_signature, int(b), int(r))
      test_similarity_matrix = get_dissimilarity_matrix(test_tvs, test_candidate_pairs, q, alpha, beta, gamma, delta, mu, epsilon)
      test_predicted_pairs = hClustering(test_similarity_matrix, best_treshold)
      test_true_pairs = get_true_duplicates(test_tvs)
      test_precision, test_recall, test_f1 = get_pr_re_F1(test_true_pairs, test_predicted_pairs, test_candidate_pairs)
      a,b, test_f1_total = get_pr_re_F1_total(test_true_pairs, test_predicted_pairs, test_candidate_pairs)

      test_comparison_fraction = len(test_candidate_pairs) / total_comparisons_test
      results_iter.append([b, r, test_comparison_fraction, test_precision, test_recall, test_f1,test_f1_total])

    results_total.extend(results_iter)

  results_total = np.array(results_total)
  results_dict = defaultdict(list)

  for row in results_total:
    b, r, test_comparison_fraction, test_precision, test_recall,test_f1, test_f1_total = row
    results_dict[(int(b), int(r))].append([test_comparison_fraction, test_precision, test_recall, test_f1,test_f1_total])

  averaged_results = []
  for (b, r), metrics in results_dict.items():
    metrics_array = np.array(metrics)
    avg_metrics = metrics_array.mean(axis=0)
    averaged_results.append([b, r] + avg_metrics.tolist())

  averaged_results = np.array(averaged_results)

  return averaged_results

## Plotting


In [269]:
def plot_lsh_results(results, results_old):
    comparison_fraction = results[:, 2]
    pq = results[:, 3]
    pc = results[:, 4]
    f1star = results[:, 5]

    comparison_fraction_old = results_old[:, 2]
    pq_old = results_old[:, 3]
    pc_old = results_old[:, 4]
    f1star_old = results_old[:, 5]

    plt.figure(figsize=(8, 6))
    plt.plot(comparison_fraction, pq, marker='o', linestyle='-', color='b', label='MSMP+B')
    plt.plot(comparison_fraction_old, pq_old, marker='x', linestyle='--', color='k', label='MSMP+')
    plt.title('Pair quality vs. Fraction of Comparisons')
    plt.xlabel('Fraction of Comparisons')
    plt.ylabel('Pair Quality')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.plot(comparison_fraction, pc, marker='o', linestyle='-', color='g', label='MSMP+B')
    plt.plot(comparison_fraction_old, pc_old, marker='x', linestyle='--', color='k', label='MSMP+')
    plt.title('Pair Completeness vs. Fraction of Comparisons')
    plt.xlabel('Fraction of Comparisons')
    plt.ylabel('Pair Completeness')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.plot(comparison_fraction, f1star, marker='o', linestyle='-', color='r', label='MSMP+B')
    plt.plot(comparison_fraction_old, f1star_old, marker='x', linestyle='--', color='k', label='MSMP+')
    plt.title('F1* vs. Fraction of Comparisons')
    plt.xlabel('Fraction of Comparisons')
    plt.ylabel('F1*')
    plt.legend()
    plt.grid(True)
    plt.show()


In [270]:
def plot_msm_results(results, results_old):
    comparison_fraction = results[:, 2]
    f1 = results[:, 5]

    comparison_fraction_old = results_old[:, 2]
    f1_old = results_old[:, 5]

    plt.figure(figsize=(10, 6))
    plt.plot(comparison_fraction, f1, marker='o', linestyle='-', color='r', label='MSMP+B')
    plt.plot(comparison_fraction_old, f1_old, marker='x', linestyle='--', color='k', label='MSMP+')
    plt.title('F1_MSM vs. Fraction of Comparisons')
    plt.xlabel('Fraction of Comparisons')
    plt.ylabel('F1_MSM')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [271]:
def plot_msm_results_total(results, results_old):
    comparison_fraction = results[:, 2]
    f1_total = results[:, 6]

    comparison_fraction_old = results_old[:, 2]
    f1_total_old = results_old[:, 6]

    plt.figure(figsize=(10, 6))
    plt.plot(comparison_fraction, f1_total, marker='o', linestyle='-', color='r', label='MSMP+B')
    plt.plot(comparison_fraction_old, f1_total_old, marker='x', linestyle='--', color='k', label='MSMP+')
    plt.title('F1_total vs. Fraction of Comparisons')
    plt.xlabel('Fraction of Comparisons')
    plt.ylabel('F1_total')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Main

In [ ]:
%%time
cleaned_data = clean_features_title('TVs-all-merged.json', 'TVs-cleaned.json')
with open(cleaned_data, 'r') as f:
    clean_data = json.load(f)
all_tvs = [item for model_id, items in clean_data.items() for item in items]
results_LSH = bootstrapLSH(all_tvs,1200,5,False)

In [ ]:
%%time
cleaned_data_old = clean_features_title('TVs-all-merged.json', 'TVs-cleaned-old.json')
with open(cleaned_data_old, 'r') as f:
    clean_data_old = json.load(f)
all_tvs_old = [item for model_id, items in clean_data_old.items() for item in items]
results_LSH_old = bootstrapLSH(all_tvs_old,1200,5,True)

In [ ]:
plot_lsh_results(results_LSH, results_LSH_old)

In [ ]:
%%time
results_MSM = bootstrapMSM(all_tvs,1200,5,False)

In [ ]:
%%time
results_MSM_old = bootstrapMSM(all_tvs_old,1200,5,True)

In [ ]:
plot_msm_results(results_MSM, results_MSM_old)

In [ ]:
%%time
results_MSM_total = bootstrapMSMtotal(all_tvs,1200,5,False)

In [ ]:
%%time
results_MSM_old_total = bootstrapMSMtotal(all_tvs_old,1200,5,True)

In [ ]:
plot_msm_results_total(results_MSM_total, results_MSM_old_total)